In [54]:
#Dependencies
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import csv

In [58]:
# find number of pages for albums 2020
url_pages = 'https://www.metacritic.com/browse/albums/score/metascore/year/filtered'
# set user agent for header
user_agent = {'User-agent': 'Mozilla/5.0'}
# send response
response_pages = requests.get(url, headers = user_agent)
# scrape website into variable to parse
soup_pages = BeautifulSoup(response_pages.text, 'html.parser')
# create temporary lists for user scores
pages = int(soup_pages.find('li', class_='page last_page').text)

# print(pages)
for page_num in range(pages):
    metaScrape(str(page_num))

In [57]:
def metaScrape(page_num):

    # request variables
    url_score = f'https://www.metacritic.com/browse/albums/score/metascore/year/filtered?page={page_num}'
    # bs4 variables
    
    # set user agent for header
    user_agent = {'User-agent': 'Mozilla/5.0'}
    # send response
    response_score = requests.get(url_score, headers = user_agent)
    # scrape website into variable to parse
    soup_score = BeautifulSoup(response_score.text, 'html.parser')
    # create temporary lists for user scores
    userP = []
    userM = []
    userN = []
    # create/initialize dictionary 
    albums_dict = {'artist':[], 'album':[], 'date':[], 'meta_score': [], 'user_score':[]}
    # csv variables
    output_path = os.path.join('..', 'data', 'albums_2020.csv')

    # create soup 
    for artist in soup.find_all('td', class_='clamp-summary-wrap'):
        # scrape album name
        albums_dict['album'].append(artist.find('a', class_= 'title').text)
        # scrape artist name and strip white space and extra characters
        albums_dict['artist'].append(artist.find('div', class_='artist').text.strip().strip("by "))
        # scrape date
        albums_dict['date'].append(artist.find('div', class_='clamp-details').find('span').text)
        # scrape meta_score, handle for changes in class name, convert data type of score to int and append to dict
        # except set to pass since all alubms have a score 
        try:
            albums_dict['meta_score'].append(int(artist.find('div', class_='metascore_w large release positive').text))  
        except:
            pass
        try:
            albums_dict['meta_score'].append(int(artist.find('div', class_='metascore_w large release mixed').text))  
        except:
            pass 
        try:
            albums_dict['meta_score'].append(int(artist.find('div', class_='metascore_w large release negative').text))  
        except:
            pass
        # scrape user score, handle errors for tbd/class name and append to temp list
        try:
            userP.append(float(artist.find('div', class_='metascore_w user large release positive').text))  
        except:
            userP.append("tbd")
        try:
            userM.append(float(artist.find('div', class_='metascore_w user large release mixed').text))  
        except:
            userM.append("tbd")
        try:
            userN.append(float(artist.find('div', class_='metascore_w user large release negative').text))  
        except:
            userN.append("tbd")
    # merge user score by filtering scores from tbd using data type in temporary lists, convert data type of scores to int and append to dictionary        
    for a, b, c in zip(userP, userM, userN):
        if isinstance(a, float):
            albums_dict['user_score'].append(int(a * 10))
        elif isinstance(b, float):
            albums_dict['user_score'].append(int(b * 10))
        elif isinstance(c, float):
            albums_dict['user_score'].append(int(c * 10))
        else:
            albums_dict['user_score'].append(c)

    # write dictionary to csv
    # create header
    fields = ['artist', 'album', 'date', 'meta_score', 'user_score'] 
    # create variable for data to be written
    data = zip(albums_dict['artist'], albums_dict['album'], albums_dict['date'], albums_dict['meta_score'], albums_dict['user_score'])
    # create file and write csv
    if page_num == 0:    
        with open(output_path, 'w') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(fields)
            for d in data:
                writer.writerow(d)
    else:
        with open(output_path, 'a') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(fields)
            for d in data:
                writer.writerow(d)

In [38]:
metaScrape()

TypeError: metaScrape() takes 0 positional arguments but 1 was given